In [ ]:
from tqdm import tqdm
import pandas as pd
import fasttext
import numpy as np

In [ ]:
model = fasttext.load_model("/home/burak/Desktop/cc.en.300.bin")

In [ ]:
df = pd.read_csv("processed.csv",encoding="utf-8",lineterminator="\n")

In [ ]:
df = df.iloc[:60000]

In [ ]:
sent_vectors = []
for item in tqdm(df.iterrows()):
    sent_vectors.append(model.get_sentence_vector(item[1].content))
    
sent_vectors = np.array(sent_vectors)

In [ ]:
labels_encoded = []
for item in tqdm(df.iterrows()):
    if item[1].label == "CYBER":
        labels_encoded.append(1)
    else:
        labels_encoded.append(0)
labels_encoded=np.array(labels_encoded)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(sent_vectors,labels_encoded,test_size=0.20)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=7)

In [ ]:
rfc.fit(X_train, Y_train)

In [ ]:
score = rfc.score(X_test, Y_test,)
score

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test, rfc.predict_proba(X_test)[:,1])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
a = precision_recall_fscore_support(Y_test, rfc.predict(X_test), average='macro')
a

In [ ]:
from sklearn.metrics import plot_confusion_matrix
%matplotlib inline
print(X_test.shape[0])
plot_confusion_matrix(rfc,X_test, Y_test)

In [ ]:
rfc.predict([model.get_sentence_vector("new version of microsoft excel is published")])

In [ ]:
import pickle
filename = 'random_forest.model'
pickle.dump(rfc, open(filename, 'wb'))